# `sklearn` `ColumnTransformer` and Text Features

## Imports

In [65]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML

sys.path.append("code/.")
from plotting_functions import *
from utils import *

pd.set_option("display.max_colwidth", 200)

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

<br><br><br><br>

## Learning outcomes 

From this lecture, you will be able to 

- use `ColumnTransformer` to build all our transformations together into one object and use it with `sklearn` pipelines;  
- define `ColumnTransformer` where transformers contain more than one steps;
- explain why text data needs a different treatment than categorical variables;
- use `scikit-learn`'s `CountVectorizer` to encode text data;
- explain different hyperparameters of `CountVectorizer`.

## sklearn's [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)

- In most applications, some features are categorical, some are continuous, some are binary, and some are ordinal. 

- When we want to develop supervised machine learning pipelines on real-world datasets, very often we want to apply different transformation on different columns. 

- Enter `sklearn`'s `ColumnTransformer`!! 

### Why we need `ColumnTransformer`
- Let's look at a toy example: 

In [2]:
df = pd.read_csv("data/quiz2-grade-toy-col-transformer.csv")
df.head(10)

,enjoy_course,ml_experience,major,class_attendance,university_years,lab1,lab2,lab3,lab4,quiz1,quiz2
0,yes,1,Computer Science,Excellent,3,92,93.0,84,91,92,A+
1,yes,1,Mechanical Engineering,Average,2,94,90.0,80,83,91,not A+
2,yes,0,Mathematics,Poor,3,78,85.0,83,80,80,not A+
3,no,0,Mathematics,Excellent,3,91,NaN,92,91,89,A+
4,yes,0,Psychology,Good,4,77,83.0,90,92,85,A+
5,no,1,Economics,Good,5,70,73.0,68,74,71,not A+
6,yes,1,Computer Science,Excellent,4,80,88.0,89,88,91,A+
7,no,0,Mechanical Engineering,Poor,3,95,93.0,69,79,75,not A+
8,no,0,Linguistics,Average,2,97,90.0,94,82,80,not A+
9,yes,1,Mathematics,Average,4,95,82.0,94,94,85,not A+


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   enjoy_course      21 non-null     object 
 1   ml_experience     21 non-null     int64  
 2   major             21 non-null     object 
 3   class_attendance  21 non-null     object 
 4   university_years  21 non-null     int64  
 5   lab1              21 non-null     int64  
 6   lab2              19 non-null     float64
 7   lab3              21 non-null     int64  
 8   lab4              21 non-null     int64  
 9   quiz1             21 non-null     int64  
 10  quiz2             21 non-null     object 
dtypes: float64(1), int64(6), object(4)
memory usage: 1.9+ KB


### Transformations on the toy data

In [4]:
df.head()

,enjoy_course,ml_experience,major,class_attendance,university_years,lab1,lab2,lab3,lab4,quiz1,quiz2
0,yes,1,Computer Science,Excellent,3,92,93.0,84,91,92,A+
1,yes,1,Mechanical Engineering,Average,2,94,90.0,80,83,91,not A+
2,yes,0,Mathematics,Poor,3,78,85.0,83,80,80,not A+
3,no,0,Mathematics,Excellent,3,91,NaN,92,91,89,A+
4,yes,0,Psychology,Good,4,77,83.0,90,92,85,A+


- Scaling on numeric features
- One-hot encoding on the categorical feature `major` and binary feature `enjoy_class`
- Ordinal encoding on the ordinal feature `class_attendance`
- Imputation on the `lab2` feature
- None on the `ml_experience` feature

### `ColumnTransformer` example

#### Data

In [5]:
X = df.drop(columns=["quiz2"])
y = df["quiz2"]
X.columns

Index(['enjoy_course', 'ml_experience', 'major', 'class_attendance',
       'university_years', 'lab1', 'lab2', 'lab3', 'lab4', 'quiz1'],
      dtype='object')

#### Identify the transformations we want to apply

In [6]:
X.head()

,enjoy_course,ml_experience,major,class_attendance,university_years,lab1,lab2,lab3,lab4,quiz1
0,yes,1,Computer Science,Excellent,3,92,93.0,84,91,92
1,yes,1,Mechanical Engineering,Average,2,94,90.0,80,83,91
2,yes,0,Mathematics,Poor,3,78,85.0,83,80,80
3,no,0,Mathematics,Excellent,3,91,NaN,92,91,89
4,yes,0,Psychology,Good,4,77,83.0,90,92,85


In [7]:
numeric_feats = ["university_years", "lab1", "lab3", "lab4", "quiz1"]  # apply scaling
categorical_feats = ["major"]  # apply one-hot encoding
passthrough_feats = ["ml_experience"]  # do not apply any transformation
drop_feats = ["lab2", "class_attendance", "enjoy_course"]  # do not include these features in modeling

For simplicity, let's only focus on scaling and one-hot encoding first. 

#### Create a column transformer

- Each transformation is specified by a name, a transformer object, and the columns this transformer should be applied to. 

In [8]:
from sklearn.compose import ColumnTransformer

In [9]:
ct = ColumnTransformer(
    [
        ("scaling", StandardScaler(), numeric_feats),
        ("onehot", OneHotEncoder(sparse=False), categorical_feats),
    ]
)

#### Convenient `make_column_transformer` syntax

- Similar to `make_pipeline` syntax, there is convenient `make_column_transformer` syntax. 
- The syntax automatically names each step based on its class. 
- We'll be mostly using this syntax. 

In [10]:
from sklearn.compose import make_column_transformer

ct = make_column_transformer(
    (StandardScaler(), numeric_feats),  # scaling on numeric features
    (OneHotEncoder(), categorical_feats),  # OHE on categorical features
    ("passthrough", passthrough_feats),  # no transformations on the binary features
    ("drop", drop_feats),  # drop the drop features
)

In [11]:
ct

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['university_years', 'lab1', 'lab3', 'lab4',
                                  'quiz1']),
                                ('onehotencoder', OneHotEncoder(), ['major']),
                                ('passthrough', 'passthrough',
                                 ['ml_experience']),
                                ('drop', 'drop',
                                 ['lab2', 'class_attendance', 'enjoy_course'])])

In [62]:
transformed = ct.fit_transform(X)


- When we `fit_transform`, each transformer is applied to the specified columns and the result of the transformations are concatenated horizontally. 
- A big advantage here is that we build all our transformations together into one object, and that way we're sure we do the same operations to all splits of the data.
- Otherwise we might, for example, do the OHE on both train and test but forget to scale the test data.

#### Let's examine the transformed data

In [13]:
type(transformed[:2])

numpy.ndarray

In [14]:
transformed

array([[-0.09345386,  0.3589134 , -0.21733442,  0.36269995,  0.84002795,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ],
       [-1.07471942,  0.59082668, -0.61420598, -0.85597188,  0.71219761,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ],
       [-0.09345386, -1.26447953, -0.31655231, -1.31297381, -0.69393613,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.09345386,  0.24295676,  0.57640869,  0.36269995,  0.45653693,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.8878117 , -1.38043616,  0.37797291,  0.51503393, -0.05478443,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.  

```{note}
Note that the returned object is not a dataframe. So there are no column names. 
```

#### Viewing the transformed data as a dataframe

- How can we view our transformed data as a dataframe? 
- We are adding more columns. 
- So the original columns won't directly map to the transformed data. 
- Let's create column names for the transformed data. 

In [15]:
column_names = (
    numeric_feats
    + ct.named_transformers_["onehotencoder"].get_feature_names().tolist()
    + passthrough_feats
)
column_names

['university_years',
 'lab1',
 'lab3',
 'lab4',
 'quiz1',
 'x0_Biology',
 'x0_Computer Science',
 'x0_Economics',
 'x0_Linguistics',
 'x0_Mathematics',
 'x0_Mechanical Engineering',
 'x0_Physics',
 'x0_Psychology',
 'ml_experience']

In [16]:
ct.named_transformers_

{'standardscaler': StandardScaler(),
 'onehotencoder': OneHotEncoder(),
 'passthrough': 'passthrough',
 'drop': 'drop'}

```{note}
Note that the order of the columns in the transformed data depends upon the order of the features we pass to the `ColumnTransformer` and can be different than the order of the features in the original dataframe.  
```

In [17]:
pd.DataFrame(transformed, columns=column_names)

,university_years,lab1,lab3,lab4,quiz1,x0_Biology,x0_Computer Science,x0_Economics,x0_Linguistics,x0_Mathematics,x0_Mechanical Engineering,x0_Physics,x0_Psychology,ml_experience
0,-0.093454,0.358913,-0.217334,0.362700,0.840028,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-1.074719,0.590827,-0.614206,-0.855972,0.712198,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,-0.093454,-1.264480,-0.316552,-1.312974,-0.693936,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-0.093454,0.242957,0.576409,0.362700,0.456537,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.887812,-1.380436,0.377973,0.515034,-0.054784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,1.869077,-2.192133,-1.804821,-2.226978,-1.844409,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.887812,-1.032566,0.278755,-0.094302,0.712198,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,-0.093454,0.706783,-1.705603,-1.465308,-1.333088,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,-1.074719,0.938697,0.774844,-1.008306,-0.693936,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.887812,0.706783,0.774844,0.819702,-0.054784,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


#### `ColumnTransformer`: Transformed data

<br>

<img src='img/column-transformer.png' width="1500">

[Adapted from here.](https://amueller.github.io/COMS4995-s20/slides/aml-04-preprocessing/#37)

#### Training models with transformed data
- We can now pass the `ColumnTransformer` object as a step in a pipeline. 

In [18]:
pipe = make_pipeline(ct, KNeighborsClassifier())
pipe.fit(X, y)
pipe.predict(X)

array(['A+', 'not A+', 'A+', 'A+', 'A+', 'not A+', 'A+', 'not A+', 'A+',
       'A+', 'A+', 'not A+', 'A+', 'A+', 'not A+', 'not A+', 'A+',
       'not A+', 'not A+', 'not A+', 'A+'], dtype=object)

### Questions for you

#### True/False: `ColumnTransformer` 

1. You could carry out cross-validation by passing a `ColumnTransformer` object to `cross_validate`. 
2. After applying column transformer, the order of the columns in the transformed data has to be the same as the order of the columns in the original data. 
3. After applying a column transformer, the transformed data is always going to be of different shape than the original data. 
4. When you call `fit_transform` on a `ColumnTransformer` object, you get a numpy ndarray. 

#### What transformations on what columns? 
Consider the feature columns below. 

- What transformations would you apply on each column? 

| colour  | location  |  shape |  water_content | weight |
|-----|-----|-----|-----|-----|
|   red   |   canada  |    NaN  |       84  |        100 |
| yellow  |   mexico  |   long  |       75  |        120 |
| orange  |   spain   |    NaN  |       90  |        NaN |
| magenta |    china  |    round|       NaN |        600 |
| purple  |  austria  |    NaN  |       80  |        115 |
| purple  |  turkey   |   oval  |       78  |        340 |
| green   |  mexico   |   oval  |       83  |        NaN |
| blue    | canada     | round  |      73   |       535  |
| brown   |  china     |   NaN  |       NaN |       1743 | 
| yellow  |  mexico    |  oval  |       83  |        265 |


<br><br><br><br>

## More on feature transformations

### Multiple transformations in a transformer

- Recall that `lab2` has missing values. 


In [19]:
X.head(10)

,enjoy_course,ml_experience,major,class_attendance,university_years,lab1,lab2,lab3,lab4,quiz1
0,yes,1,Computer Science,Excellent,3,92,93.0,84,91,92
1,yes,1,Mechanical Engineering,Average,2,94,90.0,80,83,91
2,yes,0,Mathematics,Poor,3,78,85.0,83,80,80
3,no,0,Mathematics,Excellent,3,91,NaN,92,91,89
4,yes,0,Psychology,Good,4,77,83.0,90,92,85
5,no,1,Economics,Good,5,70,73.0,68,74,71
6,yes,1,Computer Science,Excellent,4,80,88.0,89,88,91
7,no,0,Mechanical Engineering,Poor,3,95,93.0,69,79,75
8,no,0,Linguistics,Average,2,97,90.0,94,82,80
9,yes,1,Mathematics,Average,4,95,82.0,94,94,85


- So we would like to apply more than one transformations on it: imputation and scaling.  
- We can treat `lab2` separately, but we can also include it into `numeric_feats` and apply both transformations on all numeric columns.

In [20]:
numeric_feats = [
    "university_years",
    "lab1",
    "lab2",
    "lab3",
    "lab4",
    "quiz1",
]  # apply scaling
categorical_feats = ["major"]  # apply one-hot encoding
passthrough_feats = ["ml_experience"]  # do not apply any transformation
drop_feats = ["class_attendance", "enjoy_course"]

- To apply more than one transformations we can define a pipeline inside a column transformer to chain different transformations. 

In [21]:
ct = make_column_transformer(
    (make_pipeline(SimpleImputer(), StandardScaler()), numeric_feats),  # scaling on numeric features
    (OneHotEncoder(), categorical_feats),  # OHE on categorical features
    ("passthrough", passthrough_feats),  # no transformations on the binary features
    ("drop", drop_feats),  # drop the drop features
)

In [64]:
X_transformed = ct.fit_transform(X)


In [23]:
column_names = (
    numeric_feats
    + ct.named_transformers_["onehotencoder"].get_feature_names().tolist()
    + passthrough_feats
)
column_names

['university_years',
 'lab1',
 'lab2',
 'lab3',
 'lab4',
 'quiz1',
 'x0_Biology',
 'x0_Computer Science',
 'x0_Economics',
 'x0_Linguistics',
 'x0_Mathematics',
 'x0_Mechanical Engineering',
 'x0_Physics',
 'x0_Psychology',
 'ml_experience']

In [24]:
pd.DataFrame(X_transformed, columns=column_names)

,university_years,lab1,lab2,lab3,lab4,quiz1,x0_Biology,x0_Computer Science,x0_Economics,x0_Linguistics,x0_Mathematics,x0_Mechanical Engineering,x0_Physics,x0_Psychology,ml_experience
0,-0.093454,0.358913,0.893260,-0.217334,0.362700,0.840028,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-1.074719,0.590827,0.294251,-0.614206,-0.855972,0.712198,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,-0.093454,-1.264480,-0.704099,-0.316552,-1.312974,-0.693936,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-0.093454,0.242957,0.000000,0.576409,0.362700,0.456537,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.887812,-1.380436,-1.103439,0.377973,0.515034,-0.054784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,1.869077,-2.192133,-3.100139,-1.804821,-2.226978,-1.844409,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.887812,-1.032566,-0.105089,0.278755,-0.094302,0.712198,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,-0.093454,0.706783,0.893260,-1.705603,-1.465308,-1.333088,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,-1.074719,0.938697,0.294251,0.774844,-1.008306,-0.693936,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.887812,0.706783,-1.303109,0.774844,0.819702,-0.054784,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


<br><br>

### Incorporating ordinal feature `class_attendance` 

- The `class_attendance` column is different than the `major` column in that there is some ordering of the values. 
    - Excellent > Good > Average > poor    

In [25]:
X.head()

,enjoy_course,ml_experience,major,class_attendance,university_years,lab1,lab2,lab3,lab4,quiz1
0,yes,1,Computer Science,Excellent,3,92,93.0,84,91,92
1,yes,1,Mechanical Engineering,Average,2,94,90.0,80,83,91
2,yes,0,Mathematics,Poor,3,78,85.0,83,80,80
3,no,0,Mathematics,Excellent,3,91,NaN,92,91,89
4,yes,0,Psychology,Good,4,77,83.0,90,92,85


Let's try applying `OrdinalEncoder` on this column. 

In [26]:
X_toy = X[["class_attendance"]]
enc = OrdinalEncoder()
enc.fit(X_toy)
X_toy_ord = enc.transform(X_toy)
df = pd.DataFrame(data=X_toy_ord, columns=["class_attendance_enc"], index=X_toy.index)

In [27]:
pd.concat([X_toy, df], axis=1).head(10)

,class_attendance,class_attendance_enc
0,Excellent,1.0
1,Average,0.0
2,Poor,3.0
3,Excellent,1.0
4,Good,2.0
5,Good,2.0
6,Excellent,1.0
7,Poor,3.0
8,Average,0.0
9,Average,0.0


- What's the problem here? 
    - The encoder doesn't know the order. 
- We can examine unique categories manually, order them based on our intuitions, and then provide this human knowledge to the transformer. 

What are the unique categories of `class_attendance`? 

In [28]:
X_toy["class_attendance"].unique()

array(['Excellent', 'Average', 'Poor', 'Good'], dtype=object)

Let's order them manually. 

In [29]:
class_attendance_levels = ["Poor", "Average", "Good", "Excellent"]

```{note}
Note that if you use the reverse order of the categories, it wouldn't matter. 
```

Let's make sure that we have included all categories in our manual ordering.  

In [30]:
assert set(class_attendance_levels) == set(X_toy["class_attendance"].unique())

In [31]:
oe = OrdinalEncoder(categories=[class_attendance_levels], dtype=int)
oe.fit(X_toy[["class_attendance"]])
ca_transformed = oe.transform(X_toy[["class_attendance"]])
df = pd.DataFrame(data=ca_transformed, columns=["class_attendance_enc"], index=X_toy.index)
print(oe.categories_)
pd.concat([X_toy, df], axis=1).head(10)

[array(['Poor', 'Average', 'Good', 'Excellent'], dtype=object)]


,class_attendance,class_attendance_enc
0,Excellent,3
1,Average,1
2,Poor,0
3,Excellent,3
4,Good,2
5,Good,2
6,Excellent,3
7,Poor,0
8,Average,1
9,Average,1


The encoded categories are looking better now! 

#### More than one ordinal columns?

- We can pass the manually ordered categories when we create an `OrdinalEncoder` object as a list of lists. 
- If you have more than one ordinal columns
    - manually create a list of ordered categories for each column
    - pass a list of lists to `OrdinalEncoder`, where each inner list corresponds to manually created list of ordered categories for a corresponding ordinal column. 
    

Now let's incorporate ordinal encoding of `class_attendance` in our column transformer. 

In [32]:
numeric_feats = [
    "university_years",
    "lab1",
    "lab2",
    "lab3",
    "lab4",
    "quiz1",
]  # apply scaling
categorical_feats = ["major"]  # apply one-hot encoding
ordinal_feats = ["class_attendance"]  # apply ordinal encoding
passthrough_feats = ["ml_experience"]  # do not apply any transformation
drop_feats = ["enjoy_course"]  # do not include these features

In [33]:
ct = make_column_transformer(
    (make_pipeline(SimpleImputer(), StandardScaler()), numeric_feats),  # scaling on numeric features
    (OneHotEncoder(), categorical_feats),  # OHE on categorical features
    (OrdinalEncoder(categories=[class_attendance_levels], dtype=int), ordinal_feats),  # Ordinal encoding on ordinal features
    ("passthrough", passthrough_feats),  # no transformations on the binary features
    ("drop", drop_feats),  # drop the drop features
)

In [34]:
ct

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['university_years', 'lab1', 'lab2', 'lab3',
                                  'lab4', 'quiz1']),
                                ('onehotencoder', OneHotEncoder(), ['major']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['Poor', 'Average',
                                                             'Good',
                                                             'Excellent']],
                                                dtype=<class 'int'>),
                                 ['class_attendance']),
                                ('passthrough', 'passthrough',
      

In [35]:
X_transformed = ct.fit_transform(X)

In [36]:
column_names = (
    numeric_feats
    + ct.named_transformers_["onehotencoder"].get_feature_names().tolist()
    + ordinal_feats
    + passthrough_feats
)
column_names

['university_years',
 'lab1',
 'lab2',
 'lab3',
 'lab4',
 'quiz1',
 'x0_Biology',
 'x0_Computer Science',
 'x0_Economics',
 'x0_Linguistics',
 'x0_Mathematics',
 'x0_Mechanical Engineering',
 'x0_Physics',
 'x0_Psychology',
 'class_attendance',
 'ml_experience']

In [37]:
pd.DataFrame(X_transformed, columns=column_names)

,university_years,lab1,lab2,lab3,lab4,quiz1,x0_Biology,x0_Computer Science,x0_Economics,x0_Linguistics,x0_Mathematics,x0_Mechanical Engineering,x0_Physics,x0_Psychology,class_attendance,ml_experience
0,-0.093454,0.358913,0.893260,-0.217334,0.362700,0.840028,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
1,-1.074719,0.590827,0.294251,-0.614206,-0.855972,0.712198,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,-0.093454,-1.264480,-0.704099,-0.316552,-1.312974,-0.693936,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-0.093454,0.242957,0.000000,0.576409,0.362700,0.456537,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0
4,0.887812,-1.380436,-1.103439,0.377973,0.515034,-0.054784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
5,1.869077,-2.192133,-3.100139,-1.804821,-2.226978,-1.844409,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
6,0.887812,-1.032566,-0.105089,0.278755,-0.094302,0.712198,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
7,-0.093454,0.706783,0.893260,-1.705603,-1.465308,-1.333088,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,-1.074719,0.938697,0.294251,0.774844,-1.008306,-0.693936,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0.887812,0.706783,-1.303109,0.774844,0.819702,-0.054784,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0


Let us build a pipeline with the column transformer and a model.

pipeline --> columntransformer --> pipeline

In [38]:
pipe = make_pipeline(ct, KNeighborsClassifier())
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['university_years', 'lab1',
                                                   'lab2', 'lab3', 'lab4',
                                                   'quiz1']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(), ['major']),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Poor',
                                          

<center>
<img src='img/pipeline_plus_columntransformer.png' width="800">
</center> 

<br><br>

### Categorical features with only two possible categories

- Sometimes you have features with only two possible categories. 
- If we apply `OheHotEncoder` on such columns, it'll create two columns, which seems wasteful, as we could represent all information in the column in just one column with say 0's and 1's with presence of absence of one of one of the categories.
- You can pass `drop="if_binary"` argument to `OneHotEncoder` in order to create only one column in such scenario. 

In [39]:
X["enjoy_course"].head()

0    yes
1    yes
2    yes
3     no
4    yes
Name: enjoy_course, dtype: object

In [40]:
ohe_enc = OneHotEncoder(dtype=int, sparse=False)
ohe_enc.fit(X[["enjoy_course"]])
transformed = ohe_enc.transform(X[["enjoy_course"]])
df = pd.DataFrame(data=transformed, columns=["enjoy_course_enc_0", "enjoy_course_enc_1"], index=X.index)
pd.concat([X[["enjoy_course"]], df], axis=1).head(10)

,enjoy_course,enjoy_course_enc_0,enjoy_course_enc_1
0,yes,0,1
1,yes,0,1
2,yes,0,1
3,no,1,0
4,yes,0,1
5,no,1,0
6,yes,0,1
7,no,1,0
8,no,1,0
9,yes,0,1


In [41]:
ohe_enc = OneHotEncoder(drop="if_binary", dtype=int, sparse=False)
ohe_enc.fit(X[["enjoy_course"]])
transformed = ohe_enc.transform(X[["enjoy_course"]])
df = pd.DataFrame(data=transformed, columns=["enjoy_course_enc"], index=X.index)
pd.concat([X[["enjoy_course"]], df], axis=1).head(10)

,enjoy_course,enjoy_course_enc
0,yes,1
1,yes,1
2,yes,1
3,no,0
4,yes,1
5,no,0
6,yes,1
7,no,0
8,no,0
9,yes,1


In [42]:
numeric_feats = [
    "university_years",
    "lab1",
    "lab2",
    "lab3",
    "lab4",
    "quiz1",
]  # apply scaling
categorical_feats = ["major"]  # apply one-hot encoding
ordinal_feats = ["class_attendance"]  # apply ordinal encoding
binary_feats = ["enjoy_course"]  # apply one-hot encoding with drop="if_binary"
passthrough_feats = ["ml_experience"]  # do not apply any transformation
drop_feats = []

In [43]:
ct = make_column_transformer(
    (make_pipeline(SimpleImputer(), StandardScaler()), numeric_feats),  # scaling on numeric features
    (OneHotEncoder(handle_unknown="ignore"), categorical_feats),  # OHE on categorical features
    (OrdinalEncoder(categories=[class_attendance_levels], dtype=int), ordinal_feats),  # Ordinal encoding on ordinal features
    (OneHotEncoder(drop="if_binary", dtype=int), binary_feats),  # OHE on categorical features
    ("passthrough", passthrough_feats),  # no transformations on the binary features
)

In [44]:
ct

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['university_years', 'lab1', 'lab2', 'lab3',
                                  'lab4', 'quiz1']),
                                ('onehotencoder-1',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['major']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['Poor', 'Average',
                                                             'Good',
                                                             'Excellent']],
                                                dtype=<class 'int'>),
                                 [

In [45]:
pipe = make_pipeline(ct, KNeighborsClassifier())

In [46]:
scores = cross_validate(pipe, X, y, cv=5, return_train_score=True)
pd.DataFrame(scores)

,fit_time,score_time,test_score,train_score
0,0.017949,0.024934,0.80,0.812500
1,0.002809,0.015622,1.00,0.764706
2,0.015620,0.000000,0.50,1.000000
3,0.000000,0.015677,0.75,0.823529
4,0.014040,0.000000,0.75,0.941176


```{note}
Do not read too much into the scores, as we are running cross-validation on a very small dataset with 21 examples. The main point here is to show you how can we use `ColumnTransformer` to apply different transformations on different columns.
```

<br><br><br><br>

## Encoding text data  

In [47]:
toy_spam = [
    [
        "URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!",
        "spam",
    ],
    ["Lol you are always so convincing.", "non spam"],
    ["Nah I don't think he goes to usf, he lives around here though", "non spam"],
    [
        "URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!",
        "spam",
    ],
    [
        "Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030",
        "spam",
    ],
    ["Congrats! I can't wait to see you!!", "non spam"],
]
toy_df = pd.DataFrame(toy_spam, columns=["sms", "target"])

### Spam/non spam toy example 

- What if the feature is in the form of raw text?
- The feature `sms` below is neither categorical nor ordinal. 
- How can we encode it so that we can pass it to the machine learning algorithms we have seen so far? 

In [48]:
toy_df

,sms,target
0,URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!,spam
1,Lol you are always so convincing.,non spam
2,"Nah I don't think he goes to usf, he lives around here though",non spam
3,URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!,spam
4,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,spam
5,Congrats! I can't wait to see you!!,non spam


### What if we apply OHE? 

In [49]:
### DO NOT DO THIS.
enc = OneHotEncoder(sparse=False)
transformed = enc.fit_transform(toy_df[["sms"]])
pd.DataFrame(transformed, columns=enc.categories_)

,Congrats! I can't wait to see you!!,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,Lol you are always so convincing.,"Nah I don't think he goes to usf, he lives around here though",URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!,URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!
0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0


- We do not have a fixed number of categories here. 
- Each "category" (feature value) is likely to occur only once in the training data and we won't learn anything meaningful if we apply one-hot encoding or ordinal encoding on this feature. 

- How can we encode or represent raw text data into fixed number of features so that we can learn some useful patterns from it?  
- This is a well studied problem in the field of Natural Language Processing (NLP), which is concerned with giving computers the ability to understand written and spoken language. 
- Some popular representations of raw text include: 
    - **Bag of words** 
    - TF-IDF
    - Embedding representations 

### Bag of words (BOW) representation

- One of the most popular representation of raw text 
- Ignores the syntax and word order
- It has two components: 
    - The vocabulary (all unique words in all documents) 
    - A value indicating either the presence or absence or the count of each word in the document. 
        
<center>
<img src='img/surfin-bird-bow.png' width="800">
</center> 

### Extracting BOW features using `scikit-learn`
- `CountVectorizer`
    - Converts a collection of text documents to a matrix of word counts.  
    - Each row represents a "document" (e.g., a text message in our example). 
    - Each column represents a word in the vocabulary (the set of unique words) in the training data. 
    - Each cell represents how often the word occurs in the document.       

```{note}
In the Natural Language Processing (NLP) community text data  is referred to as a **corpus** (plural: corpora). ```   

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X_counts = vec.fit_transform(toy_df["sms"])
bow_df = pd.DataFrame(X_counts.toarray(), columns=vec.get_feature_names(), index=toy_df["sms"])
bow_df

,08002986030,100000,11,900,always,are,around,as,been,call,...,update,urgent,usf,valued,wait,week,with,won,you,your
sms,,,,,,,,,,,,,,,,,,,,,
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!,0,0,0,1,0,0,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
Lol you are always so convincing.,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
"Nah I don't think he goes to usf, he lives around here though",0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,1,0
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,1,0,1,0,0,0,0,0,0,1,...,2,0,0,0,0,0,1,0,0,1
Congrats! I can't wait to see you!!,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [51]:
type(toy_df["sms"])

pandas.core.series.Series

```{important}
Note that unlike other transformers we are passing a `Series` object to `fit_transform`. For other transformers, you can define one transformer for more than one columns. But with `CountVectorizer` you need to define separate `CountVectorizer` transformers for each text column, if you have more than one text columns.    
```

In [52]:
X_counts

<6x61 sparse matrix of type '<class 'numpy.int64'>'
	with 71 stored elements in Compressed Sparse Row format>

### Why sparse matrices? 

- Most words do not appear in a given document.
- We get massive computational savings if we only store the nonzero elements.
- There is a bit of overhead, because we also need to store the locations:
    - e.g. "location (3,27): 1".
    
- However, if the fraction of nonzero is small, this is a huge win.

In [53]:
print("The total number of elements: ", np.prod(X_counts.shape))
print("The number of non-zero elements: ", X_counts.nnz)
print("Proportion of non-zero elements: %0.4f" % (X_counts.nnz / np.prod(X_counts.shape)))
print("The value at cell (3,%d) is: %d" % (vec.vocabulary_["jackpot"], X_counts[3, vec.vocabulary_["jackpot"]]))

The total number of elements:  366
The number of non-zero elements:  71
Proportion of non-zero elements: 0.1940
The value at cell (3,27) is: 1



Question for you
- What would happen if you apply scaling, e.g., `StandardScaler` on sparse data? 

Centering sparse data would destroy the sparseness structure in the data, and thus rarely is a sensible thing to do.

### `OneHotEncoder` and sparse features 
- By default, `OneHotEncoder` also creates sparse features. 
- You could set `sparse=False` to get a regular `numpy` array. 
- If there are a huge number of categories, it may be beneficial to keep them sparse.
- For smaller number of categories, it doesn't matter much.

### Important hyperparameters of `CountVectorizer` 

- `binary`
    - whether to use absence/presence feature values or counts. If True, all non zero counts are set to 1
- `max_features`
    - only consider top `max_features` ordered by frequency in the corpus
- `max_df`
    - ignore features which occur in more than `max_df` documents 
- `min_df` 
    - ignore features which occur in less than `min_df` documents 
- `ngram_range`
    - consider word sequences in the given range 
<center>
<img src='img/ngram.png' width="800">
</center> 

Let's look at all features, i.e., words (along with their frequencies).

In [54]:
vec = CountVectorizer()
X_counts = vec.fit_transform(toy_df["sms"])
bow_df = pd.DataFrame(X_counts.toarray(), columns=vec.get_feature_names(), index=toy_df["sms"])
bow_df

,08002986030,100000,11,900,always,are,around,as,been,call,...,update,urgent,usf,valued,wait,week,with,won,you,your
sms,,,,,,,,,,,,,,,,,,,,,
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!,0,0,0,1,0,0,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
Lol you are always so convincing.,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
"Nah I don't think he goes to usf, he lives around here though",0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,1,0
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,1,0,1,0,0,0,0,0,0,1,...,2,0,0,0,0,0,1,0,0,1
Congrats! I can't wait to see you!!,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


When we use `binary=True`, the representation uses presence/absence of words instead of word counts.   

In [55]:
vec_binary = CountVectorizer(binary=True)
X_counts = vec_binary.fit_transform(toy_df["sms"])
bow_df = pd.DataFrame(X_counts.toarray(), columns=vec_binary.get_feature_names(), index=toy_df["sms"])
bow_df

,08002986030,100000,11,900,always,are,around,as,been,call,...,update,urgent,usf,valued,wait,week,with,won,you,your
sms,,,,,,,,,,,,,,,,,,,,,
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!,0,0,0,1,0,0,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
Lol you are always so convincing.,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
"Nah I don't think he goes to usf, he lives around here though",0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,1,0
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,1,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,1
Congrats! I can't wait to see you!!,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


We can control the size of X (the number of features) using `max_features`.

In [56]:
vec8 = CountVectorizer(max_features=8)
X_counts = vec8.fit_transform(toy_df["sms"])
bow_df = pd.DataFrame(X_counts.toarray(), columns=vec8.get_feature_names(), index=toy_df["sms"])
bow_df

,free,have,mobile,the,to,update,urgent,you
sms,,,,,,,,
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!,0,1,0,0,1,0,1,1
Lol you are always so convincing.,0,0,0,0,0,0,0,1
"Nah I don't think he goes to usf, he lives around here though",0,0,0,0,1,0,0,0
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!,1,1,0,0,0,0,1,1
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,2,0,2,2,2,2,0,0
Congrats! I can't wait to see you!!,0,0,0,0,1,0,0,1


### Preprocessing

- Note that `CountVectorizer` is carrying out some preprocessing such as because of the default argument values 
    - Converting words to lowercase (`lowercase=True`)
    - getting rid of punctuation and special characters (`token_pattern ='(?u)\\b\\w\\w+\\b'`)


### Build a knn model for text

In [57]:
pipe = make_pipeline(CountVectorizer(), KNeighborsClassifier())

In [58]:
pipe.fit(toy_df["sms"], toy_df["target"])

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [59]:
pipe.predict(toy_df["sms"])

array(['non spam', 'non spam', 'non spam', 'non spam', 'spam', 'non spam'],
      dtype=object)

### Is this a realistic representation of text data? 

- Of course this is not a great representation of language
    - We are throwing out everything we know about language and losing a lot of information. 
    - It assumes that there is no syntax and compositional meaning in language.  
- But it works surprisingly well for many tasks. 
- We will learn more expressive representations in the coming weeks. 

<br><br>

### Questions for you

#### `CountVectorizer`: True or False

1. As you increase the value for `max_features` hyperparameter of `CountVectorizer` the training score is likely to go up. 
2. Suppose you are encoding text data using `CountVectorizer`. If you encounter a word in the validation or the test split that's not available in the training data, we'll get an error. 
3. `max_df` hyperparameter of `CountVectorizer` can be used to get rid of most frequently occurring words from the dictionary.
4. In the code below, inside `cross_validate`, each fold might have slightly different number of features (columns) in the fold.

```
pipe = (CountVectorizer(), KNeighborsClassifier())
cross_validate(pipe, X_train, y_train)
```

#### Identify column transformations

Consider the restaurant data from the survey you did a few weeks ago. 

In [60]:
restaurant_data = pd.read_csv("data/cleaned_restaurant_data.csv")
restaurant_data.head(10)

,north_america,eat_out_freq,age,n_people,price,food_type,noise_level,good_server,comments,restaurant_name,target
0,Yes,3.0,29,20,10,Other,no music,Yes,NaN,NaN,like
1,Yes,2.0,23,10,20,Chinese,low,Yes,food tastes good,Midam,like
2,Yes,2.0,21,10,40,Other,low,Yes,good food,pear tree,like
3,No,2.0,24,15,40,Other,medium,Yes,My love for Korean food,Dami,like
4,Yes,5.0,23,20,10,Canadian/American,low,Yes,Good food,NaN,like
5,Yes,2.0,22,60,20,Chinese,medium,Yes,food tasted great!,NaN,like
6,Yes,2.0,23,10000000,1000000,Canadian/American,crazy loud,Yes,NaN,CACTUS CLUB CAFE,like
7,Yes,4.0,20,20,40,Chinese,high,Yes,NaN,NaN,like
8,Yes,1.0,21,40,60,Italian,high,Yes,NaN,Frankie’s,like
9,No,2.0,20,50,45,Canadian/American,medium,Yes,"Steak was good, ambiance was nice and server was helpful.",Hy's steakhouse,like


What all feature transformations you would apply on this dataset? 

<br><br><br><br>

## What did we learn today?

- Motivation to use `ColumnTransformer`
- `ColumnTransformer` syntax
- Defining transformers with multiple transformations
- How to visualize transformed features in a dataframe 
- More on ordinal features 
- Different arguments `OneHotEncoder`
    - `handle_unknow="ignore"`
    - `if_binary`
- Dealing with text features
    - Bag of words representation: `CountVectorizer`